In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
df_solar = pd.read_csv('SolarSummaries.csv')
df_solar[:3]

In [1]:
df_sun = df_solar[['ZIP', 'State','Annual Average Minimum (kWh/m2/day)']]
df_sun.columns = ['ZIP','State','Sun']
df_sun['ZIP'] = df_sun['ZIP'].apply(lambda x: str(x).zfill(5))
df_sun[:3]

NameError: name 'df_solar' is not defined

In [ ]:
df_census = pd.read_csv('ACS_13_5YR_DP04_with_ann.csv',skiprows=1,low_memory = False)
df_census[:3]

In [ ]:
df_housing = df_census[['Id2','Estimate; UNITS IN STRUCTURE - Total housing units - 1-unit, detached']]
df_housing.columns = ['ZIP','Units']
df_housing['ZIP'] = df_housing['ZIP'].apply(lambda x: str(x).zfill(5))
df_housing[:3]

In [ ]:
df_byzip = df_sun.merge(df_housing)
df_byzip[:5]
df_byzip["Units"].sum()

In [ ]:
print 'max sun value is ' + str(df_byzip['Sun'].max()) + '; min ' + str(df_byzip['Sun'].min()) + '; mean ' + str(df_byzip['Sun'].mean())
print 'max units/ZIP is ' + str(df_byzip['Units'].max()) + '; min ' + str(df_byzip['Units'].min()) + '; mean ' + str(df_byzip['Units'].mean())

In [ ]:
# capacity is in gigawatts
df_capacity = pd.read_csv('state_capacity_80pct_RE_ETI.csv')
df_capacity[:50]

In [ ]:
# generation is in gigawatt-hours
df_generation = pd.read_csv('state_generation_80pct_RE_ETI.csv')
df_generation[:3]

In [ ]:
# don't feel like typing out the US state abbreviations
df_states= pd.read_csv('state_table.csv')
df_states[:3]

In [ ]:
df_states = df_states[['name','abbreviation']]
df_states.columns = ['state', 'State']
df_states[:3]

In [ ]:
df_capacity = df_capacity.merge(df_states)[['State','year','rooftop_pv']]
df_capacity[:50]

In [ ]:
# unit_capacity = 0.010 # gigawatt-hours
# df_generation['units_needed'] = df_generation['rooftop_pv'] / unit_capacity # units_needed = state_capacity / unit_capacity
# df_generation['units_fraction'] = df_generation['units_needed'] / # units_fraction = units_needed / units_state
# calculating unit_capacity:
# 1.5 * 6000 / 250 → 6kW, a big home solar system in CA → 36 m^2

unit_capacity = df_byzip['Sun'].mean()*365*.15*30/1000 # in MW-hour per year
print 'a large home solar system will produce ' + str(unit_capacity) + ' MW-hours per year'

In [ ]:
df_capacity['num_systems'] = df_capacity['rooftop_pv']*1e6/5e3
df_capacity[:5]

In [ ]:
# need to sum up units per state as well
print 'number of units in Alaska: ' + str(df_byzip[df_byzip.State == 'AL']['Units'].sum())
units_state = {}
for state in df_byzip.State.drop_duplicates().tolist():
    units_state[state] = df_byzip[df_byzip.State == state]['Units'].sum()
units_state.items()[:5]